In [1]:
# import json
# import re
# from docx import Document

# # Đường dẫn tới file .docx
# file_path = r'data\trich_dan_luat\docx\Luật_Quốc hội_10-2022-QH15_Nội vụ_Chính quyền địa phương_10-11-2022_01-07-2023_thực hiện dân chủ ở cơ sở.docx'
# # Đường dẫn để lưu file .json
# output_json_path = r'data\trich_dan_luat\docx\output.json'

# # Hàm để kiểm tra số la mã
# def is_roman_numeral(s):
#     return bool(re.match(r"^(I|V|X|L|C|D|M)+$", s))

# # Đọc file .docx
# document = Document(file_path)

# # Tạo danh sách để lưu trữ kết quả
# results = []

# # Biến để lưu trữ thông tin về Chương, Mục, Tiểu mục hiện tại
# current_chapter = None
# current_section = None
# current_mini_section = None

# # Duyệt qua từng đoạn văn trong tài liệu
# for i in range(len(document.paragraphs)):
#     paragraph_text = document.paragraphs[i].text.strip()

#     # Kiểm tra xem đoạn văn có bắt đầu bằng "Chương" và theo sau là một số la mã
#     match_chapter = re.match(r'^Chương (\w+)', paragraph_text)
#     if match_chapter and is_roman_numeral(match_chapter.group(1)):
#         current_chapter = f"Chương {match_chapter.group(1)}"
#         continue

#     # Kiểm tra xem đoạn văn có bắt đầu bằng "Mục"
#     if paragraph_text.startswith("Mục"):
#         current_section = paragraph_text
#         continue

#     # Kiểm tra xem đoạn văn có bắt đầu bằng "Tiểu mục"
#     if paragraph_text.startswith("Tiểu mục"):
#         current_mini_section = paragraph_text
#         continue

#     # Kiểm tra xem đoạn văn có bắt đầu bằng "Điều"
#     if paragraph_text.startswith("Điều"):
#         article_entry = {
#             "Chapter": current_chapter,
#             "Section": current_section,
#             "Mini-Section": current_mini_section,
#             "Article": paragraph_text,
#             "Content": ""
#         }
#         content = []

#         # Duyệt qua các đoạn văn tiếp theo cho đến khi gặp "Chương", "Mục", "Tiểu", "Điều"
#         for j in range(i + 1, len(document.paragraphs)):
#             next_paragraph = document.paragraphs[j].text.strip()
            
#             # Kiểm tra điều kiện dừng
#             if next_paragraph.startswith(("Chương", "Mục", "Tiểu", "Điều")):
#                 break
            
#             # Thêm đoạn văn vào nội dung
#             content.append(next_paragraph)

#         # Nối các đoạn văn thành một chuỗi
#         article_entry["Content"] = "\n".join(content)
#         results.append(article_entry)  # Thêm vào danh sách kết quả

# # Lưu kết quả vào file .json
# with open(output_json_path, 'w', encoding='utf-8') as json_file:
#     json.dump(results, json_file, ensure_ascii=False, indent=4)

# print(f"Đã lưu thành công vào file: {output_json_path}")

In [7]:
import json
from docx import Document

# Đường dẫn tới file .docx
file_path = r'data\trich_dan_luat\docx\Luật_Quốc hội_10-2022-QH15_Nội vụ_Chính quyền địa phương_10-11-2022_01-07-2023_thực hiện dân chủ ở cơ sở.docx'
# Đường dẫn để lưu file .json
output_json_path = r'data\trich_dan_luat\docx\output.json'

# Đọc file .docx
document = Document(file_path)

# Tạo danh sách để lưu trữ kết quả
results = []

# Biến để lưu trữ thông tin về Chương, Mục, Tiểu mục hiện tại
current_chapter = None
current_section = None
current_mini_section = None

# Duyệt qua từng đoạn văn trong tài liệu
for i in range(len(document.paragraphs)):
    paragraph_text = document.paragraphs[i].text.strip()

    # Kiểm tra xem đoạn văn có bắt đầu bằng "Chương"
    if paragraph_text.startswith("Chương"):
        current_chapter = paragraph_text  # Cập nhật giá trị cho chapter
        current_section = None  # Reset Mục
        current_mini_section = None  # Reset Tiểu mục
        # Kiểm tra đoạn văn tiếp theo
        if i + 1 < len(document.paragraphs):
            next_paragraph = document.paragraphs[i + 1].text.strip()
            if not next_paragraph.startswith(("Mục", "Tiểu mục", "Điều")):
                current_chapter += f": {next_paragraph}"  # Nối đoạn tiếp theo vào Chapter
        continue

    # Kiểm tra xem đoạn văn có bắt đầu bằng "Mục"
    if paragraph_text.startswith("Mục"):
        current_section = paragraph_text
        current_mini_section = None  # Reset Tiểu mục khi có Mục mới
        continue

    # Kiểm tra xem đoạn văn có bắt đầu bằng "Tiểu mục"
    if paragraph_text.startswith("Tiểu mục"):
        current_mini_section = paragraph_text
        continue

    # Kiểm tra xem đoạn văn có bắt đầu bằng "Điều"
    if paragraph_text.startswith("Điều"):
        article_entry = {
            "Chapter": current_chapter,  # Giá trị sẽ là đoạn text bắt đầu bằng "Chương" + đoạn tiếp theo
            "Section": current_section if current_section else None,  # Gán None nếu không có Mục
            "Mini-Section": current_mini_section if current_mini_section else None,  # Gán None nếu không có Tiểu mục
            "Article": paragraph_text,
            "Content": ""
        }
        content = []

        # Duyệt qua các đoạn văn tiếp theo cho đến khi gặp "Chương", "Mục", "Tiểu", "Điều"
        for j in range(i + 1, len(document.paragraphs)):
            next_paragraph = document.paragraphs[j].text.strip()
            
            # Kiểm tra điều kiện dừng
            if next_paragraph.startswith(("Chương", "Mục", "Tiểu", "Điều")):
                break
            
            # Thêm đoạn văn vào nội dung
            content.append(next_paragraph)

        # Nối các đoạn văn thành một chuỗi
        article_entry["Content"] = "\n".join(content)
        results.append(article_entry)  # Thêm vào danh sách kết quả

# Lưu kết quả vào file .json
with open(output_json_path, 'w', encoding='utf-8') as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4)

print(f"Đã lưu thành công vào file: {output_json_path}")

Đã lưu thành công vào file: data\trich_dan_luat\docx\output.json
